<a href="https://colab.research.google.com/github/DonRomaniello/CitibikeDocks/blob/master/Monthly_CSV_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import time

import os
import tarfile

import json

In [54]:
#Set initial directory as the directory where the script is running.

initial_directory = '.'
os.chdir(initial_directory)
print("Current working directory is", os.getcwd())

Current working directory is /Users/donromaniello/Google Drive/Citibike/TARS/ToBeProcessed/TimeDirectories/2021/Feb-2021


In [ ]:
# Select directories

directory_prompt_selection = 'none'

while directory_prompt_selection != 'w':
  current_directories = [d for d in os.listdir('.') if os.path.isdir(d)]

  print("Current working directory is", os.getcwd())

  for directory in current_directories:

    print(current_directories.index(directory), directory)

  print("w", "We can use current directory")
  print("c", "Change top level directory")
  print("u", "Navigate up one level.")
  directory_prompt_selection = input("Choose an option:")
  print("\n")
  
  if directory_prompt_selection == 'c':
    directory_prompt_selection = input('Provide either the relative or full path name of the directory')
    os.chdir(directory_prompt_selection)
  elif directory_prompt_selection == 'u':
    os.chdir('../')
  elif directory_prompt_selection == 'w':
    None
  else:
    os.chdir(os.path.join('.', current_directories[int(directory_prompt_selection)]))

print("Current working directory is", os.getcwd())

In [ ]:
# Get rid of anything the operating system may add...
files = ([x for x in sorted(filter(os.path.isfile,
                                   os.listdir('.')),
                                   key=os.path.getmtime) if x[-2:] == "xz"])

files

In [ ]:
stopwatch_start = time.time()

files_by_month = [x for x in sorted(filter(os.path.isfile,
                                          os.listdir('.')),
                                    key=os.path.getmtime) if x[-2:] == "xz"]

hour = tarfile.open(directory + '/' + files_by_month[0])
hour_list = hour.getnames()

minute_json = json.load(hour.extractfile(hour_list[0]))
timestamp = minute_json["last_updated"]
stations = (pd.DataFrame(minute_json['data']['stations'])
                [['station_id', 'num_docks_available']]
                .rename(columns={'num_docks_available':timestamp})
                .astype('int16')
                .set_index('station_id')
                .sort_index()[timestamp])

minute = 1
while minute < len(hour_list):
  try:
    minute_json = json.load(hour.extractfile(hour_list[minute]))
    timestamp = minute_json["last_updated"]
    stations = pd.concat([stations, 
                        (pd.DataFrame(minute_json['data']['stations'])
                          [['station_id', 'num_docks_available']]
                          .rename(columns={'num_docks_available':timestamp})
                          .astype('int16')
                          .set_index('station_id')
                          .sort_index()[timestamp])],
                        join='outer',
                        axis=1)
    
    minute+=1
  except json.JSONDecodeError:
    minute+=1
        

for hour_xz in files_by_month[1:]:
  hour = tarfile.open(directory + '/' + hour_xz)
  hour_list = hour.getnames()

  minute=0
  while minute < len(hour_list):
    try:
      minute_json = json.load(hour.extractfile(hour_list[minute]))
      timestamp = minute_json["last_updated"]
      stations = pd.concat([stations, 
                        (pd.DataFrame(minute_json['data']['stations'])
                          [['station_id', 'num_docks_available']]
                          .rename(columns={'num_docks_available':timestamp})
                          .astype('int16')
                          .set_index('station_id')
                          .sort_index()[timestamp])],
                        join='outer',
                        axis=1)
      minute+=1
    except json.JSONDecodeError:
      minute+=1

stopwatch_end = time.time()
print(stopwatch_end - stopwatch_start)

4099.887139797211


In [ ]:
stations.to_csv(('./' + os.getcwd().split('/')[-1] + 'csv.gzip'))

In [ ]:
save_test = pd.read_csv('/Users/donromaniello/Google Drive/Citibike/Processed/Jan-2020.csv.gzip')

In [ ]:
print(save_test.head())

Int64Index([  72,   79,   82,   83,  116,  119,  120,  127,  128,  143,
            ...
            3902, 3903, 3905, 3907, 3909, 3910, 3911, 3916, 3917, 3918],
           dtype='int64', name='station_id', length=939)